In [1]:
# Dependencies:

#!pip install transformers ipywidgets IPython

# If running on Colab:
#!git clone https://github.com/sam-paech/antislop-sampler.git
#!mv antislop-sampler/src .
#!mv antislop-sampler/slop_phrase_prob_adjustments.json .

In [2]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
from src.antislop_generate import generate_antislop, chat_antislop

# Enable efficient transfer for Hugging Face models
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = "1"

# Set the device to 'cuda' if available, else 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Specify the model name (replace with your preferred model)
model_name = "unsloth/Llama-3.2-1B-Instruct"
#model_name = "unsloth/Llama-3.2-3B-Instruct"

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
#model.pad_token_id = tokenizer.eos_token_id
model.to(device)
print('Model loaded')

Model loaded


In [3]:
import json
if os.path.exists('slop_phrase_prob_adjustments.json'):
    with open('slop_phrase_prob_adjustments.json', 'r') as f:
        slop_phrase_prob_adjustments = dict(json.load(f)[:500])

In [4]:
prompt = "Write a story about Elara, the weaver of tapestries in future Technopolis. In the bustling city, a group of "

In [5]:
# Chat generation with streaming
messages = [
    {"role": "user", "content": prompt}
]
for token in chat_antislop(
    model=model,
    tokenizer=tokenizer,
    messages=messages,
    max_new_tokens=400,
    # Antislop sampling may be less reliable at low temperatures.
    temperature=1,    
    min_p=0.1,
    # The adjustment_strength param scales how strongly the probability adjustments are applied.
    # A value of 1 means the values in slop_phrase_prob_adjustments (or the defaults) are used unmodified.
    # Reasonable values are 0 (disabled) thru 100+ (effectively banning the list).
    adjustment_strength=100.0,
    # Optional: Provide a list of slop phrases and probability adjustments
    slop_phrase_prob_adjustments=slop_phrase_prob_adjustments,
    enforce_json=False,
    antislop_enabled=True,
    streaming=True
):
    print(tokenizer.decode(token), end='', flush=True)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


In the heart of Future Polis, where skyscrapers pierced the sky and neon lights danced across the streets, a small, unassuming shop stood out among the crowds. "Moonwhisper's Textiles" was the name, and "weaver of tapestries" was the nickname that had been whispered about for generations.

Inside the shop, the air was alive with the hum of looms and the soft whisper of threads as they wove themselves into the fabric of reality. Among the rows of intricately patterned tapestries, one woman stood out - a young weaver named Eliana, known to her friends and customers as simply "The Weaver".

Eliana's fingers moved with a precision that belied her age, her hands weaving and knotting threads into a living, breathing narrative that told stories of love, loss, and redemption. Her tapestries were more than just beautiful works of art - they were portals to other worlds, gateways to memories yet to be forged.

One day, as the sun set over the city, a group of strangers entered the shop, their fa

In [6]:
# Chat generation without streaming
messages = [
    {"role": "user", "content": prompt}
]
generated_text = chat_antislop(
    model=model,
    tokenizer=tokenizer,
    messages=messages,
    max_length=400,
    temperature=1,
    min_p=0.1,
    adjustment_strength=100.0,
    slop_phrase_prob_adjustments=slop_phrase_prob_adjustments,
    enforce_json=False,
    antislop_enabled=True,
    streaming=False
)
print(tokenizer.decode(generated_text))

In the heart of future Terraverde, a city of unparalleled innovation and technological advancement, a small, unassuming shop stood on a narrow street lined with sleek skyscrapers. The sign above the door read "Weaver of Dreams," and the windows were crowded with an assortment of peculiar items: intricately carved stone pedestals, glowing orbs that pulsed with an ethereal light, and delicate filigree threads that seemed to shimmer with an otherworldly energy.

This was the domain of a young woman named Lyra, a skilled weaver of tapestries with a reputation for creating the most breathtaking works of art in all of Terraverde. Her fingers danced across the loom, weaving a mesmerizing pattern of light and shadow that seemed to capture the very essence of the city itself.

Lyra's shop was a sanctuary for the people of Terraverde, a place where they could come to escape the hum of technology and the constant din of commerce. Inside, the air was thick with the scent of wool and lavender, and 

In [7]:
# generate without streaming
prompt_with_template = tokenizer.apply_chat_template(messages, tokenize=False)
generated_text = generate_antislop(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_length=300,
    temperature=1,
    min_p=0.1,
    adjustment_strength=100.0,
    slop_phrase_prob_adjustments=slop_phrase_prob_adjustments,
    enforce_json=False,
    antislop_enabled=True,
    streaming=False
)        
print(tokenizer.decode(generated_text))

 urchins have begun to notice that the weavers have been quietly adding strange symbols to their own textiles.

At first, the urchins think it's just a prank or a weird art trend, but as the symbols become more prominent and seem to be telling a different story, they start to wonder if there's something more sinister at work.

The urchin leader, a scrappy kid named Kael, decides to investigate further and discovers that the weavers have been adding symbols that seem to be connected to a long-forgotten event from the city's history. As Kael delves deeper into the mystery, he uncovers a dark secret that the weavers are hiding.

The symbols on the tapestries are actually a warning, left by a group of rebels who fought against the corrupt ruling elite in the past. They have been trying to communicate with the city's inhabitants for generations, but their voices have been silenced by the ruling elite.

Kael realizes that the weavers are not just adding symbols, they are actually trying to t

In [8]:
# generate with streaming
prompt_with_template = tokenizer.apply_chat_template(messages, tokenize=False)
for token in generate_antislop(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_length=300,
    temperature=1,
    min_p=0.1,
    slop_phrase_prob_adjustments=slop_phrase_prob_adjustments,
    adjustment_strength=100.0,
    enforce_json=False,
    antislop_enabled=True,
    streaming=True
):
    print(tokenizer.decode(token), end='', flush=True)

 urchins sneak out to watch the celestial ballet of stars in the night sky, a tradition that has been a part of the city's culture for centuries. They find a lone weaver, who is working on a new piece of fabric, and strike up a conversation. The weaver, a mysterious woman with an air of quiet determination, listens to their tale of adventure and bravery. She shares a secret of her own, one that only a select few are privy to.

In the city, the weaver's skills as a weaver and a seamstress are highly sought after, as the wealthy elite use her wares to adorn their palaces and cloaks. But in the undercity, where the streets are narrow and the alleys are dark, the true value of the weaver's art lies in the hidden patterns and secret symbols that are woven into every thread. It is here that the urchins and the weaver's secret meet, and a bond is forged between them. The weaver reveals to the urchins a way to see the stars in a new and wondrous way, using the patterns and symbols that are wov